# [Deploy and Run] Dploy the Trained Custom Speech Models
This sample demonstrates how to deploy and run the trained Custom Speech models calling REST API. 

> ✨ ***Note*** <br>
> Except for batch transcription, you must deploy a custom endpoint to use a custom speech model. You can deploy an endpoint for a base or custom model, and then update the endpoint later to use a better trained model. Endpoints used by F0 Speech resources are deleted after seven days.

## Prerequisites
Configure a Python virtual environment for 3.10 or later: 
 1. open the Command Palette (Ctrl+Shift+P).
 1. Search for Python: Create Environment.
 1. select Venv / Conda and choose where to create the new environment.
 1. Select the Python interpreter version. Create with version 3.10 or later.

## Setup the environment

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import os
import json
from openai import AzureOpenAI
import requests
from dotenv import load_dotenv
from utils.common import *

load_dotenv()

SPEECH_KEY = os.getenv("AZURE_AI_SPEECH_API_KEY")
SPEECH_REGION = os.getenv("AZURE_AI_SPEECH_REGION")
CUSTOM_SPEECH_LANG = os.getenv("CUSTOM_SPEECH_LANG")
CUSTOM_SPEECH_LOCALE = os.getenv("CUSTOM_SPEECH_LOCALE")

# Get the project and custom model IDs from the previous notebook
project_id = ""
custom_model_with_plain_id = ""
custom_model_with_acoustic_id = ""
%store -r project_id
%store -r custom_model_with_plain_id
%store -r custom_model_with_acoustic_id

try:
    project_id, custom_model_with_plain_id, custom_model_with_acoustic_id
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the previous notebook again.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

print(project_id, custom_model_with_plain_id, custom_model_with_acoustic_id)

## 1. Deploy a trained speech model
- In order to get more information about deploying the custom models, check this document link below
- https://learn.microsoft.com/en-us/azure/ai-services/speech-service/how-to-custom-speech-deploy-model?pivots=rest-api

In [ ]:
import requests
import time
import json

# Base URL for the Speech Services REST API
base_url = f'https://{SPEECH_REGION}.api.cognitive.microsoft.com/speechtotext'

# Headers for authentication
headers = {
    'Ocp-Apim-Subscription-Key': SPEECH_KEY,
    'Content-Type': 'application/json'
}

### Check the custom speech model ids to deploy

In [ ]:
# Set the custom model id to evaluate
print(custom_model_with_plain_id, custom_model_with_acoustic_id)

# Set the model id to deploy
deploy_model_id = custom_model_with_acoustic_id

## Create an endpoint to deploy the custom speech model

In [ ]:

display_name = f"[{CUSTOM_SPEECH_LANG}] My custom model deployment"
description = "Custom model endpoint to deloy"
logging_enabled = True
endpoint_id = create_endpoint(base_url, headers, project_id, deploy_model_id, display_name, description, CUSTOM_SPEECH_LOCALE, logging_enabled)

In [ ]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_endpoint_status(endpoint_id):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = get_endpoint_status(base_url, headers, endpoint_id)
        if status == "NotStarted":
            pbar.update(1)
        while status != "Succeeded" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = get_endpoint_status(base_url, headers, endpoint_id)
        while(pbar.n < 3):
            pbar.update(1)
        print("Endpoint Creation Completed")

In [ ]:
monitor_endpoint_status(endpoint_id)

## 2. Test the deployed custom model 

- Test your deployed custom model by Python SDK. In order to set the endpoint, you need to get the endpoint id from the previous step.

In [ ]:
def speech_recognition_from_file_with_endpoint(file_path: str, lang:str, endoint_id: str):
    # Check the Parameters - https://learn.microsoft.com/en-us/python/api/azure-cognitiveservices-speech/azure.cognitiveservices.speech.speechconfig?view=azure-python
    speech_config = speechsdk.SpeechConfig(subscription=SPEECH_KEY, region=SPEECH_REGION, speech_recognition_language=lang)
    speech_config.endpoint_id = endoint_id
    audio_config = speechsdk.AudioConfig(filename=file_path)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    speech_recognition_result = speech_recognizer.recognize_once_async().get()
    return speech_recognition_result.text

### Get the sorted wav files from the dataset folder

In [ ]:
import os
from IPython.display import Audio, display

output_folder = 'synthetic_eval_data'
files = os.listdir(output_folder)
wav_files = [file for file in files if file.endswith('.wav')]

# Sort wav_files by 'no' in ascending order
wav_files.sort(key=lambda x: int(x.split('_')[0]))
wav_files

In [ ]:
for wav_file in wav_files[0:3]:
    print(speech_recognition_from_file_with_endpoint(os.path.join(output_folder,wav_file), CUSTOM_SPEECH_LOCALE, endpoint_id))

## 3. Delete the all resources after testing
- Don't forget to delete the deployed custom model and project after testing.

In [ ]:
delete_endpoint(base_url, headers, endpoint_id)

In [ ]:
delete_project(base_url, headers, project_id)